<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/creating_chord_collection.git


from creating_chord_collection.collection.resources.scales import scales
from creating_chord_collection.collection.resources.transposable_figures import transposable_figures



In [2]:



# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'creating_chord_collection']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world


from creating_chord_collection.collection.chord import GuitarChord

from creating_chord_collection.collection.collection import ChordCollection







print(write_imports([]))






In [3]:
# @title Chord


# Example
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
print(chord)
print("String Frequencies:", chord.calculate_frequencies())
print("Is Open Chord:", chord.is_open())
chord.transpose(0)
print(chord.get_notes(include_strings=True))
print(chord.get_notes())

# Validate the chord against the C major scale
ionian_scale = scales["ionian"]
tonic = "C"
is_chord_in_scale = chord.validate_against_scale(tonic, ionian_scale)
print(f"Chord fits in C major scale: {is_chord_in_scale}")


('E', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
String Frequencies: {1: 329.63, 2: 246.94, 3: 196.0, 4: 164.81110255326524, 5: 123.47082531403103, 6: 82.41}
Is Open Chord: True
{1: 'E', 2: 'B', 3: 'G', 4: 'E', 5: 'B', 6: 'E'}
['E', 'B', 'G']
Chord fits in C major scale: True


In [5]:
# @title AbstractBuilder
class AbstractBuilder:
    def __init__(self):
        self.image_size = None
        self.frets_endpoints = None
        self.strings_endpoints = None
        self.notes_coordenates = None
        self.finger_ascending = None
        self.is_scale = None
        self.font = None
        self.font_size = None
        self.starting_fret = None
        self.scale = None

    def draw_image(self):
        raise NotImplementedError

    def draw_frets(self):
        raise NotImplementedError

    def draw_strings(self):
        raise NotImplementedError

    def write_starting_fret(self):
        raise NotImplementedError

    def write_name(self, name):
        raise NotImplementedError

    def draw_notes(self, notes_style):
        raise NotImplementedError

    def get_result(self):
        raise NotImplementedError


In [6]:
# @title Concrete Builders
class ShortChordBuilder(AbstractBuilder):
    # Implement specific methods

class LongChordBuilder(AbstractBuilder):
    # Implement specific methods

class ShortScaleBuilder(AbstractBuilder):
    # Implement specific methods

class LongScaleBuilder(AbstractBuilder):
    # Implement specific methods


IndentationError: expected an indented block after class definition on line 1 (<ipython-input-6-280338935432>, line 4)

In [7]:
# @title Director
class Director:
    def __init__(self, builder):
        self._builder = builder

    def build_chord(self, chord):
        self._builder.draw_image()
        self._builder.draw_frets()
        self._builder.draw_strings()
        self._builder.write_starting_fret()
        self._builder.write_name(chord.root + " " + chord.chord_type)
        self._builder.draw_notes("chord")

    def build_scale(self, scale):
        self._builder.draw_image()
        self._builder.draw_frets()
        self._builder.draw_strings()
        self._builder.write_starting_fret()
        self._builder.write_name(scale.name)
        self._builder.draw_notes("scale")


In [8]:
# @title Execution
# Example usage
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)

# Creating a concrete builder
builder = ShortChordBuilder()

# Creating a director with the builder
director = Director(builder)

# Building the chord
director.build_chord(chord)

# Retrieving the result
result = builder.get_result()


NameError: name 'ShortChordBuilder' is not defined